In [35]:
# dependencies
import os
import glob
import pandas as pd
import numpy as np
from default.matplotlib_ import *
import sympy as sy

In [42]:
class greenhouse(object):
    
    def __init__(self): # global variables for greenhouse structure
        self.Aside_singlespan = sideHeight * length * 2 * roofCount
        self.Aside_multispan = sideHeight * length * 2
        self.Aupperbehind = np.pi * roofArchLongRadius * roofArchShortRadius/2 * roofCount * 2
        self.Alowerbehind = width * sideHeight * 2 * roofCount
        self.Aroof = np.pi * ((5 * (roofArchLongRadius + roofArchShortRadius)/4) - ((roofArchLongRadius * roofArchShortRadius)/(roofArchLongRadius + roofArchShortRadius))) * length * roofCount * 0.5
        self.As = width * length * roofCount # Afloor
        self.Ag = np.sum([self.Aside_singlespan, self.Aside_multispan, self.Aupperbehind, self.Alowerbehind, self.Aroof])

# init params
class gr_energy(greenhouse):
    def __init__(self):
        greenhouse.__init__(self)
        self.ht = float(df_ht.loc[df_ht["film_type"] == "glass", ["param"]].values)
        self.fr = float(df_fr.loc[df_fr["insulation_film"] == "multiple_insulation_curtain", ["param"]].values)
        self.hv = float(df_hv.loc[df_hv["gr_type"] == "glass", ["param"]].values)
        self.hs = 1 # default
        self.eps_roofFIR = 0.85
        self.eps_roofNIR = 0.13
        self.eps_air = 1
        self.Froof_air = 1
        # self.Ag = Ag
        # self.As = As
        self.fw = 1
        self.Ai = self.Ag/self.As
        self.rho = 5.67 * 10 ** -8 # stefan boltzmann const W/m 2 K 4

    def Qq(self, qt, qv):
        print("Greenhouse Heating load is calculated ")
        return (self.Ag * (qt + qv) + self.As) * self.fw
        #return (Ag * (qt + qv) + As * qs) * fw # 수경재배 시 토양전열량 제외 
        """
        Qq: 최대난방부하량 (kcal h-1)
        qr: 피복면적당 관류전열손실량(kcal m-2 h -1)
        qv: 단위피복면적당 틈새환기 열손실량 (kcal m-2 h-1)
        qs: 단위 상면적(바닥면적)당 토양전열량 (kcal m-2 h-1)
        Ag: 온실피복면적 (m2), As: 온실상면적 (바닥면적) (m2)
        fw: 풍속보정계수 (일반지역의 온실 1.0, 강풍지역 온실 1.1)
        """
    def qt(self, Ts, Td):
        return self.ht * (Ts - Td) * (1 - self.fr)
        """
        ht: 관류열전달계수 (kcal m-2 h-2 °C-1)
        Ts: 난방설정온도 (°C)
        Td: 설계외기온도 (°C)
        fr: 보온피복의 열절감률
        """
    def qv(self, Ts, Td):
        return self.hv * (Ts - Td)
        """hv: 환기전열계수 """
    def qs(self, Ts, Td):
        return self.hs * (Ts - Td)
        """hs: 토양전열계수 """

In [75]:
class plant():
    def __init__(self, commonname, variety, LAI):
        self.commonname = commonname
        self.variety = variety
        self.DAT = 0
        self.LAI = LAI
        print("{0} plant status created for initialize simulation".format(self.commonname))

    def progress(self, sim_duration):
        self.DAT += sim_duration
        print("{0} simulated for {1} days".format(self.name, self.DAT))


vapor = []

class water(object):

    def __init__(self):
        # transpiration parameters
        self.G = 0 # heat flux in soil [MJ m-2 dia -1]
        self.u2 = 1 # wind velocity in greenhouse
        self.gamma =  0.054 #psychrometric constant [kPa ◦C −1 ].
        

    def vapor_pressure(self, temperature, RH):
        delta = 2504 * np.exp(17.27 * temperature / (temperature + 273.2)) / (temperature + 237.3) ** 2 # Tetens (1930), Murray (1967) # differential of sy.exp(20.386 - 5132/x)
        es = 610.7 * 10 ** ((7.5 * temperature)/(237.3 + temperature)) / 1000 # 
        ea = 610.7 * 10 ** ((7.5 * temperature)/(237.3 + temperature)) / 1000 * (1 - RH/100)
        vapor.append([delta, es, ea])
        temp_df = pd.DataFrame(vapor[0]).T
        temp_df.columns = ['delta', 'es', 'ea']
        print("Return vapor pressure parameters dataframe delta, es, ea")
        return temp_df

    def transpiration(self, temperature, Rn, delta, es, ea): # PM model https://www.mdpi.com/2073-4395/12/11/2593
        return (0.408 * delta * (Rn - self.G) + (self.gamma * (900 / (temperature + 273)) * self.u2 * (es - ea))) \
            /(delta + self.gamma * (1 + 0.34 * self.u2)) #rn = J/cm/h -> MJ/m2/h 

class CO2enrichment(plant):
    def __init__(self):
        plant.__init__(self, "tomato", "Dafnis", "3")
        
class carbontracker(object):
    def __init__(self):
        self.name = "carbontracker"     

    def heating_CO2(self, heatdemand):
        return heatdemand / 860 / 3 * 0.5 # unit kg CO2   
    """"""
    # 1Kwh = 860kcal
    # 1kwh 전기에 0.5 Kg 의 CO2가 발생하는것으로 가정
    # 히트펌프 소비전력 * 히트펌프 소비계수 // 전기난방 이외는 따로 계산 필요
    """"""
    # def nutrient(self, waterconsumption, fert_n, fert_p, fert_k): ## each nutrient to carbon parameters
    #     CO2 = (fert_n)* 2.49 * 10 ** -2
    #     CH4 = fert_n * + fert_p * + fert_k + 
    #     N2O = fert_n * + fert_p * + fert_k + 
    #     return

    
    # fig = plt.figure(figsize=((8/2.54*2.5), (6/2.54*3)))
    # gs =  gridspec.GridSpec(nrows=4, ncols=1, figure=fig)
    # ax1 = plt.subplot(gs[0])
    # ax1.plot(env_df.datetime, a.es- a.ea, c = cmap[3]) # vpd
    # ax2 = plt.subplot(gs[1])
    # ax2.plot(env_df.datetime, a.delta, c = cmap[4])
    # ax3 = plt.subplot(gs[2])
    # ax3.plot(env_df.datetime, env_df["RAD(MJ/m2)"], c = cmap[0])
    # ax4 = plt.subplot(gs[3])
    # ax4.plot(env_df.datetime, trn, c = cmap[5])

In [44]:
# 10수염
nut_obj = {"N_NO3": 224.0, "N_NH4": 16.8, "P": 46.4, "K": 371.5, "Ca": 216.5, "Mg": 58.3, "S": 141.2, "pH": 0, "type": 10}
nut_water = {"N_NO3": 2.4, "N_NH4": 0, "P": 0.2, "K": 0, "Ca": 28.7, "Mg": 3.8, "S": 0, "pH": 7.8, "type": 0}
nut_lack = {key: nut_obj[key] - nut_water.get(key, 0) for key in nut_obj}

class nutrientcalc(object):
    def __init__(self):
        
        self.type = nut_lack["type"] # 10수염 or 4수염 of CaNO3
        self.N_NO3_l = nut_lack['N_NO3'] #lack
        self.N_NH4_l = nut_lack['N_NH4']
        self.P_l = nut_lack['P']
        self.K_l = nut_lack['K']
        self.Ca_l = nut_lack['Ca']
        self.Mg_l = nut_lack['Mg']
        self.S_l = nut_lack['S']
        self.pH = nut_lack['pH'] * -1
        pass
    
    def get_indiv_nutrients(self):
        
        KH2PO4_P = self.P_l if self.type == 10 else self.P_l - NH4H2PO4_P
        KH2PO4_K = KH2PO4_P / 0.22761 * 0.28731

        if self.type == 10: #NH4H2PO4_NH4_N
            NH4H2PO4_NH4_N = 0
        else:
            if self.P_l / 0.2693 * 12.177 > self.N_NH4_l:
                NH4H2PO4_NH4_N = self.N_NH4_l
            else:
                NH4H2PO4_NH4_N = self.P_l
        NH4H2PO4_P = NH4H2PO4_NH4_N / 0.12177 * 0.2693

        if self.type == 10:
            CaNO310H2O_Ca = self.Ca_l
        else:
            CaNO310H2O_Ca = 0
        
        CaNO310H2O_NO3_N = CaNO310H2O_Ca / 0.18544 * 0.14258
        CaNO310H2O_NH4_N = CaNO310H2O_Ca / 0.18544 * 0.01296
        
        MgSO47H2O_Mg = self.Mg_l # - MgNO36H2O_Mg
        MgSO47H2O_S =  MgSO47H2O_Mg / 0.09861 * 0.1301

        # MgNO36H2O_NO3_N
        # MgNO36H2O_Mg

        NH4NO3_NH4_N = self.N_NH4_l - CaNO310H2O_NH4_N - NH4H2PO4_NH4_N if self.N_NH4_l - CaNO310H2O_NH4_N - NH4H2PO4_NH4_N > 0 else 0
        NH4NO3_NO3_N = NH4NO3_NH4_N 

        if self.pH < 7:
            HNO3_NO3_N = 0
        elif self.pH >= 7 and self.Ca_l <= 20 and self.K_l < 60: 
            HNO3_NO3_N = 1.33
        elif self.pH >= 7 and self.Ca_l <= 25:
            HNO3_NO3_N = 2.668
        else:
            HNO3_NO3_N = 2.668      
        
        CaNO34H2O_Ca = 0 if CaNO310H2O_NO3_N > 0 else self.Ca_l
        CaNO34H2O_NO3_N = CaNO34H2O_Ca / 0.16971 * 0.11863

        KNO3_NO3_N = self.N_NO3_l - CaNO34H2O_NO3_N - CaNO310H2O_NO3_N -  NH4NO3_NO3_N - HNO3_NO3_N if self.N_NO3_l - CaNO34H2O_NO3_N - CaNO310H2O_NO3_N -  NH4NO3_NO3_N - HNO3_NO3_N >= 0 else 0
        KNO3_K = KNO3_NO3_N / 0.13854 * 0.38672

        K2SO4_K = 0 if KH2PO4_K + KNO3_K >= self.K_l else self.K_l - KH2PO4_K - KNO3_K
        K2SO4_S = K2SO4_K / 0.4486 * 0.18401
        
        KH2PO4 = KH2PO4_P / 0.22761
        NH4H2PO4 = NH4H2PO4_NH4_N / 0.12177
        KNO3 = KNO3_NO3_N / 0.13854
        K2SO4 = K2SO4_K / 0.4486
        CaNO34H2O = CaNO34H2O_Ca / 0.16971
        CaNO310H2O = CaNO310H2O_Ca / 0.18544
        MgSO47H2O = MgSO47H2O_Mg / 0.09861
        MgNO36H2O = 0 # not used for Mg ion
        NH4NO3 = NH4NO3_NH4_N / 0.17499
        HNO3 = HNO3_NO3_N / 0.22228

        return KH2PO4, NH4H2PO4, KNO3, K2SO4, CaNO34H2O, CaNO310H2O, MgSO47H2O, MgNO36H2O, NH4NO3, HNO3


## Simulation

In [45]:
class simulation(gr_energy):
    def __init__(self):
        gr_energy.__init__(self)
        self.Td = env_df.temperature
        self.Ts = env_df.temp_objective
        self.qt_calc = gr_energy().qt(self.Ts, self.Td)
        self.qv_calc = gr_energy().qv(self.Ts, self.Td)
        self.Qq_calc = gr_energy().Qq(self.qt_calc, self.qv_calc)
        self.CO2 = self.Qq_calc / 860 / 3 * 0.5 # unit kg CO2
        self.Qq_CO2 = np.where(self.CO2<0, 0, self.CO2)
        
    def plot(self):
        
        fig = plt.figure(figsize=((8/2.54*2.5), (6/2.54*3)))
        axes = []
        LABEL =  ['qt', 'qv', 'Qq(Kcal/h)', 'CO2 emissions (Kg CO2 per h)']

        gs =  gridspec.GridSpec(nrows=4, ncols=1, figure=fig)
        for _ in range(4):
            axes.append(plt.subplot(gs[_]))

        for ax in axes:
            ax.spines['right'].set_visible(False)
            ax.spines['left'].set_position(('outward', 10))
            ax.spines['bottom'].set_position(('outward', 5))

        axes[0].plot(env_df.datetime, self.qt_calc, alpha=0.5, c = cmap[0])
        axes[1].plot(env_df.datetime, self.qv_calc, alpha=0.5, c = cmap[1])
        axes[2].plot(env_df.datetime, self.Qq_calc, alpha=0.5, c = cmap[2])
        axes[3].plot(env_df.datetime, self.Qq_CO2, alpha=0.5, c = cmap[3])

        for _ in range(4):
            axes[_].set_ylabel(LABEL[_])

        ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
        fig.tight_layout()

        plt.show()

    def sim_results(self):
        return self.Qq_calc
        
def simulation_start():
    print("[Information] Starting simulation")
    
def simulation_end():
    print("[Information] Finished simulation")

    

### Define the variables

In [46]:
dict_design = {'roofCount': 5, 'width': 7, 'sideHeight': 1.6, 'height': 3.5, 'length': 20, 'roofArchLongRadius': 3.5, 'roofArchShortRadius': 1.9}
for design_user_input, value in dict_design.items():
    globals()[design_user_input] = value

indiv_nutrients = {"KH2PO4": 0, "NH4H2PO4": 0, "KNO3": 0, "K2SO4": 0, "Ca(NO3)24H2O": 0, "5[Ca(NO3)22H2O]NH4NO3": 0,\
    "MgSO47H2O": 0, "Mg(NO3)26H2O": 0, "NH4NO3": 0, "NH4NO3": 0, "HNO3": 0} # initialize variables
# ---------------------------------------------------------------- 온실외부 환경변수 입력

DIR_env = './data/envobs/'
FILES = [_ for _ in os.listdir(DIR_env) if _.endswith('.csv')]
FILES.sort()

for FILE in FILES:
    ## indexer = FILE.split('.')[0][3:]
    temp = pd.read_csv(f'{DIR_env}/{FILE}')
    temp.columns = ['Idx','place','datetime','RH', 'temperature' , 'Air_velo' ,'RAD(MJ/m2)']
    temp["datetime"] = pd.to_datetime(temp["datetime"])
    temp["temp_objective"] = 18 * np.ones(len(temp.temperature))
    temp = temp.loc[:, ["datetime", 'temperature', 'temp_objective', 'RAD(MJ/m2)', 'RH']]
    env_df = temp

# parameters
df_ht = pd.DataFrame({'film_type': ["glass", "glass_2layer", "plastic_1layer"],
                'param': [5.2, 2.8, 5.65]})
df_fr = pd.DataFrame({'insulation_film': ["glass_2layer", "PE_2layer", "multiple_insulation_curtain"],
                'param': [0.35, 0.45, 0.55]})
df_hv = pd.DataFrame({'gr_type': ["glass", "plastic", "closed"],
                'param': [0.4, 0.3, 0]})

# -----------------------------------------------------------------------------온실 내부 환경변수 및 제어값 입력

In [147]:
def EC_to_TDS(EC):
    # 1ds/m = 1 mmho/cm 
    # TDS (mg/L or ppm)
    # The ratio of TDS to EC of various salt solutions ranges from 550 to 700 ppm per dS/m, depending on the compositions of the solutes in the water. 
    # For soil extracts in the EC range from 3 to 30 dS/m, TDS can be estimated using the formula below:The US Salinity Laboratory (1954) also used the following empirical relationship between EC and the total soluble salt concentration (TSS, mmol/L)
    if EC > 0.1 and EC < 5:
        return EC * 640
    elif EC >= 5:
        return EC * 800
    else:
        raise ValueError("Set EC is too low < (EC == 0.1)")


def Indivnutrient_to_carbon():
    pass

simulation_start()

plant("Tomato", "Dafnis", "3")

gr_hl = gr_energy().Qq(gr_energy().qv(env_df['temp_objective'], env_df['temperature']), gr_energy().qt(env_df['temp_objective'], env_df['temperature'])) # heat load calculation
vp_params = water().vapor_pressure(env_df["temperature"], env_df["RH"])
water_use = water().transpiration(env_df["temperature"], env_df["RAD(MJ/m2)"], vp_params['delta'], vp_params['ea'], vp_params['es'])
water_use_quant = water_use.sum() * greenhouse().As
ion_temp = list(nutrientcalc().get_indiv_nutrients())

for i, value in enumerate(indiv_nutrients):
    indiv_nutrients[value] = ion_temp[i]

simulation_end()

print("---Greenhouse Heating Load: {0} kcal/year".format(gr_hl.sum()))
print("---Water Use: {0} mm/dia/year".format(water_use.sum()))
print("---Water Use Quant: {0} L".format(water_use_quant))

print("---set_nutrient concentration: {0} (mg/L)".format(EC_to_TDS(3)))
print("---set_nutrient solution: {0}, (unit: mg/L)".format(indiv_nutrients))




[Information] Starting simulation
Tomato plant status created for initialize simulation
Greenhouse Heating load is calculated 
Return vapor pressure parameters dataframe delta, es, ea
[Information] Finished simulation
---Greenhouse Heating Load: 154114618.35954067 kcal/year
---Water Use: 3994.282513524092 mm/dia/year
---Water Use Quant: 2795997.759466864 L
---set_nutrient concentration: 1920 (mg/L)
---set_nutrient solution: {'KH2PO4': 202.97877949123497, 'NH4H2PO4': 0.0, 'KNO3': 511.49409975321305, 'K2SO4': 257.19386671380016, 'Ca(NO3)24H2O': 0.0, '5[Ca(NO3)22H2O]NH4NO3': 1012.7264883520277, 'MgSO47H2O': 552.6822837440421, 'Mg(NO3)26H2O': 0, 'NH4NO3': 21.00156986660793, 'HNO3': 12.002879251394639}, (unit: mg/L)


In [157]:
# 개별 비료 생산에 드는 CO2 factor
nutrientsolutions_cf = {'KH2PO4': 202.97877949123497, 'NH4H2PO4': 0.0, 'KNO3': 511.49409975321305, 'K2SO4': 257.19386671380016, 'Ca(NO3)24H2O': 0.0, '5[Ca(NO3)22H2O]NH4NO3': 1012.7264883520277, \
    'MgSO47H2O': 552.6822837440421, 'Mg(NO3)26H2O': 0, 'NH4NO3': 21.00156986660793, 'HNO3': 12.002879251394639}